In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from imblearn.over_sampling import SMOTE,RandomOverSampler

In [2]:
test_df = pd.read_csv("./test.csv",parse_dates=['Datetime'])

In [8]:
test_df['Hour'] = test_df['Datetime'].apply(lambda time: time.hour)
test_df['Month'] = test_df['Datetime'].apply(lambda time: time.month)
test_df['Day of Week'] = test_df['Datetime'].apply(lambda time: time.dayofweek)

In [9]:
test_df.isna().sum()

ID                       0
Datetime                 0
Sensor1_PM2.5            0
Sensor2_PM2.5            0
Temperature          88090
Relative_Humidity    88090
Hour                     0
Month                    0
Day of Week              0
dtype: int64

In [10]:
missing_values_df = test_df[test_df['Temperature'].isna()]

In [11]:
missing_values_df

,ID,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Hour,Month,Day of Week
1,ID_U2EKAY4SGA94,2022-02-16 21:30:17,56.03,52.18,NaN,NaN,21,2,2
6,ID_HF47YBS9WXCH,2022-02-08 18:38:13,38.50,45.10,NaN,NaN,18,2,1
7,ID_S8EW0HOMWYRL,2022-03-04 01:24:42,30.60,27.27,NaN,NaN,1,3,4
9,ID_U59U3D19H38F,2022-02-19 19:36:29,37.98,31.88,NaN,NaN,19,2,5
11,ID_097E1YY0XGKP,2022-03-13 16:33:59,12.20,29.58,NaN,NaN,16,3,6
...,...,...,...,...,...,...,...,...,...
127356,ID_HNCS9Q2GNCQB,2022-02-13 00:37:14,33.65,42.23,NaN,NaN,0,2,6
127357,ID_GIIQYTE11L3H,2022-03-09 16:26:42,21.23,15.72,NaN,NaN,16,3,2
127358,ID_EC7F1P4GXKCL,2022-02-18 14:47:28,41.45,47.25,NaN,NaN,14,2,4
127359,ID_J64RVG2BRAG0,2022-03-11 19:14:11,28.60,27.27,NaN,NaN,19,3,4


In [12]:
mean_values = test_df.groupby(['Month','Day of Week',"Hour"]).mean()

In [13]:
mean_values

Sensor1_PM2.5  Sensor2_PM2.5  Temperature  \
Month Day of Week Hour                                              
1     0           0         74.171185      71.998519         22.7   
                  1         48.705625      47.628281         23.1   
                  2         42.723182      41.755985         23.1   
                  3         59.902519      59.611704         23.1   
                  4         58.348797      57.760752         21.9   
...                               ...            ...          ...   
3     6           19        27.240316      31.184032          NaN   
                  20        31.300752      35.306195          NaN   
                  21        32.180979      35.003106          NaN   
                  22        27.896240      31.139341          NaN   
                  23        26.188969      28.961450          NaN   

                        Relative_Humidity  
Month Day of Week Hour                     
1     0           0                  80.0  
                  1                  79.0  
                  2                  81.0  
                  3                  82.0  
                  4                  85.0  
...                                   ...  
3     6           19                  NaN  
                  20                  NaN  
                  21                  NaN  
                  22                  NaN  
                  23                  NaN  

[504 rows x 4 columns]

In [14]:
hour = dict(missing_values_df["Hour"])

hour_list = list(hour.values())

day = dict(missing_values_df["Day of Week"])

day_list = list(day.values())

month = dict(missing_values_df["Month"])

month_list = list(month.values())
    

gotcha_temp = []
gotcha_hum = []

for i in range(0,len(day_list)):
    gotcha_temp.append(mean_values.loc[month_list[i],day_list[i],hour_list[i]]['Temperature'])

month_list[0],day_list[0],hour_list[0]

(2, 2, 21)

In [15]:
for i in range(0,len(day_list)):
    gotcha_hum.append(mean_values.loc[month_list[i],day_list[i],hour_list[i]]['Relative_Humidity'])

len(gotcha_hum)

missing_values_df['Temperature'] = gotcha_temp

<ipython-input-15-e6a00f304120>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_values_df['Temperature'] = gotcha_temp


In [16]:
missing_values_df['Relative_Humidity'] = gotcha_hum



<ipython-input-16-dc01528f317f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_values_df['Relative_Humidity'] = gotcha_hum


In [17]:
missing_values_df.corr()


,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Hour,Month,Day of Week
Sensor1_PM2.5,1.000000,0.887552,-0.243619,0.294298,-0.135260,-0.249908,-0.136234
Sensor2_PM2.5,0.887552,1.000000,-0.233595,0.280798,-0.129587,-0.188148,-0.137758
Temperature,-0.243619,-0.233595,1.000000,-0.910632,0.393664,-0.180046,0.044934
Relative_Humidity,0.294298,0.280798,-0.910632,1.000000,-0.418589,0.207308,-0.070331
Hour,-0.135260,-0.129587,0.393664,-0.418589,1.000000,-0.018606,-0.025257
Month,-0.249908,-0.188148,-0.180046,0.207308,-0.018606,1.000000,0.006710
Day of Week,-0.136234,-0.137758,0.044934,-0.070331,-0.025257,0.006710,1.000000


In [19]:
missing_values_df.isna().sum()

ID                       0
Datetime                 0
Sensor1_PM2.5            0
Sensor2_PM2.5            0
Temperature          44745
Relative_Humidity    44745
Hour                     0
Month                    0
Day of Week              0
dtype: int64

In [21]:
month_keys = list(month.keys())

In [22]:
for i in month_keys:
    test_df.loc[i,'Temperature'] = missing_values_df.loc[i,'Temperature']
    test_df.loc[i,'Relative_Humidity'] = missing_values_df.loc[i,'Relative_Humidity']

In [23]:
test_df.isna().sum()

ID                       0
Datetime                 0
Sensor1_PM2.5            0
Sensor2_PM2.5            0
Temperature          44745
Relative_Humidity    44745
Hour                     0
Month                    0
Day of Week              0
dtype: int64

In [24]:
test_df.to_csv("new_test.csv")